In [2]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from collections import defaultdict

# Constants representing the paths to the datasets
offer_retailer_csv = 'offer_retailer.csv'
brand_category_csv = 'brand_category.csv'
categories_csv = 'categories.csv'

In [9]:
class OfferSearchTool:
    def __init__(self):
        self.merged_df = self.load_datasets()
        

    def load_datasets(self):
        # Load the datasets into pandas DataFrames
        offers_df = pd.read_csv(offer_retailer_csv)
        brands_df = pd.read_csv(brand_category_csv)
        categories_df = pd.read_csv(categories_csv)

        # Merge the datasets on the appropriate keys
        merged_df = pd.merge(offers_df, brands_df, on='BRAND', how='left')
        merged_df = pd.merge(merged_df, categories_df, left_on='BRAND_BELONGS_TO_CATEGORY', right_on='PRODUCT_CATEGORY', how='left')

        # If there are any missing values after the merge, you can fill them with a placeholder
        merged_df.fillna('Unknown', inplace=True)
        print(merged_df.head())
        return merged_df

    def search(self, query, query_type):
    # Use defaultdict to handle potential duplicate offers with their highest score
        results = defaultdict(lambda: {'score': 0, 'row_data': None})

    # Iterate over each row in the merged DataFrame
        for _, row in self.merged_df.iterrows():
        # Initialize the similarity score
            similarity = 0

        # Calculate the similarity based on the query type
            if query_type == 'category':
                similarity = fuzz.partial_ratio(query.lower(), row['PRODUCT_CATEGORY'].lower())
            elif query_type == 'brand':
                similarity = fuzz.partial_ratio(query.lower(), row['BRAND'].lower())
            elif query_type == 'retailer':
                similarity = fuzz.partial_ratio(query.lower(), row['RETAILER'].lower())
            elif query_type == 'offer':
                similarity = fuzz.partial_ratio(query.lower(), row['OFFER'].lower())

        # Update results if this is the highest score for this offer so far
            if similarity > results[row['OFFER']]['score']:
                results[row['OFFER']]['score'] = similarity
                results[row['OFFER']]['row_data'] = row

    # If no strong matches found and the query type is 'category', search within the broader categories
        if query_type == 'category' and all(res['score'] <= 80 for res in results.values()):
            for _, row in self.merged_df.iterrows():
                if 'IS_CHILD_CATEGORY_TO' in row and query.lower() in row['IS_CHILD_CATEGORY_TO'].lower():
                # Use a fixed similarity score for broader category matches
                    parent_category_similarity = 75  # Lower score for broader category matches
                    if parent_category_similarity > results[row['OFFER']]['score']:
                        results[row['OFFER']]['score'] = parent_category_similarity
                        results[row['OFFER']]['row_data'] = row

    # Convert results to a DataFrame, ensuring that row data is not None
        results_df = pd.DataFrame([res['row_data'] for res in results.values() if res['row_data'] is not None])
        results_df['Search Score'] = [res['score'] for res in results.values() if res['row_data'] is not None]

    # Sort by 'Search Score' and return the top 50
        results_df.sort_values(by='Search Score', ascending=False, inplace=True)
        return results_df.head(50)


    def run_cli(self):
        valid_query_types = ['brand', 'category', 'retailer']
        while True:
            query_type = input("Enter the query type ('brand', 'category', 'retailer' or 'exit' to stop): ").lower()
            if query_type == 'exit':
                break
            
            if query_type not in valid_query_types:
                print("Query should be one of the following: 'brand', 'category', 'retailer'.")
                continue

            query = input("Enter your search query: ").lower()
            if query == 'exit':
                break

            # Call the search method with both query and query_type
            results_df = self.search(query, query_type)

            if results_df.empty:
                print("No results found.\n\n")
                continue
            
            # Now we print the rows from results_df, not results
            for index, result in results_df.iterrows():
                print(f"Offer: {result['OFFER']}, Score: {result['Search Score']}")
            print("\n---\n\n")
                
# The main execution point of the script
if __name__ == "__main__":
    tool = OfferSearchTool()
    tool.run_cli()

                                            OFFER   RETAILER        BRAND  \
0  Spend $50 on a Full-Priced new Club Membership  SAMS CLUB    SAMS CLUB   
1    Beyond Meat® Plant-Based products, spend $25    Unknown  BEYOND MEAT   
2    Beyond Meat® Plant-Based products, spend $25    Unknown  BEYOND MEAT   
3    Beyond Meat® Plant-Based products, spend $25    Unknown  BEYOND MEAT   
4        Good Humor Viennetta Frozen Vanilla Cake    Unknown   GOOD HUMOR   

  BRAND_BELONGS_TO_CATEGORY RECEIPTS                           CATEGORY_ID  \
0                   Unknown  Unknown                               Unknown   
1          Plant-Based Meat   1584.0  d8cb60e5-b0c6-478a-971d-c6c55b17831f   
2   Frozen Plant-Based Meat    313.0  8e0a9431-5462-4810-9f65-68fe36adf454   
3             Packaged Meat     30.0  e73f7957-0e65-4466-9588-795bdc5f67ac   
4           Frozen Desserts   1052.0  38160828-1029-4505-9849-673773c5fad3   

          PRODUCT_CATEGORY IS_CHILD_CATEGORY_TO  
0                 

Enter the query type ('brand', 'category', 'retailer' or 'exit' to stop): exit
